In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 1
!hostname
!pwd

dv002.bridges2.psc.edu
/ocean/projects/asc170022p/mtragoza/mre-pinn/MICCAI-2023


In [2]:
import sys, os
import numpy as np
import pandas as pd

sys.path.append('..')
%aimport mre_pinn

sys.path.append('../../param_search')
%aimport param_search
ps = param_search

Using backend: pytorch



# MICCAI 2023 patient experiment

In [3]:
%pwd

'/ocean/projects/asc170022p/mtragoza/mre-pinn/MICCAI-2023'

In [18]:
# define the job template and name format

template = '''\
#!/bin/bash
#SBATCH --job-name={job_name}
#SBATCH --account=asc170022p
#SBATCH --partition=GPU-shared
#SBATCH --gres=gpu:1
#SBATCH --time=48:00:00
#SBATCH -o %J.stdout
#SBATCH -e %J.stderr
#SBATCH --mail-type=all

hostname
pwd
source activate MRE-PINN

python ../../../train.py \\
    --xarray_dir {xarray_dir} \\
    --example_id {example_id} \\
    --frequency {frequency} \\
    --omega {omega} \\
    --polar_input {polar_input} \\
    --pde_name {pde_name} \\
    --pde_warmup_iters 10000 \\
    --pde_step_iters 5000 \\
    --pde_step_factor 10 \\
    --pde_init_weight {pde_init_weight} \\
    --pde_loss_weight {pde_loss_weight} \\
    --a_loss_weight {a_loss_weight} \\
    --anatomical {anatomical} \\
    --save_prefix {job_name}    
'''
name = 'train_{data_name}_{example_id}_{omega}_{pde_name}_{a_loss_weight:.0e}'

# define the parameter space

param_space = ps.ParamSpace(
    data_name='patient',
    xarray_dir='../../../data/NAFLD/v4',
    example_id=['0006', '0020', '0024', '0029', '0043', '0047', '0126', '0135', '0155', '0164', '0172', '0173', '0210', '0213', '0219', '0222', '0234', '0235', '0291', '0344', '0378', '0384', '0395', '0401', '0415', '0457', '0461', '0491', '0492', '0496', '0509', '0510', '0516', '0525', '0564', '0604', '0612', '0615', '0628', '0639', '0648', '0650', '0653', '0655', '0659', '0693', '0704', '0717', '0727', '0734', '0737', '0740', '0747', '0748', '0898', '0900', '0929', '0931', '0932', '0940', '0954', '0964', '0965', '0979', '0989', '0995', '1028', '1033', '1039', '1045', '1075', '1076', '1077', '1083', '1103', '1106', '1108', '1110', '1119', '1121', '1123', '1134', '1144', '1149', '1205', '1207', '1215', '1217', '1230', '1261', '1271', '1296', '1303', '1311', '1329', '1337', '1341', '1343', '1344', '1382', '1395', '1400', '1404', '1417', '1426', '1427', '1433', '1435', '1448', '1453', '1456', '1464', '1671', '1679', '1694', '1699', '1706', '1712', '1714', '1715', '1720', '1722', '1727', '1736', '1748', '1785', '1786', '1790', '1791', '1793', '1795', '1798', '1806', '1807', '1819', '1829', '1839', '1843', '1851', '1853', '1896', '1899', '1903', '1917', '1935', '1940', '1948', '1967', '1979', '1980', '2001', '2007', '2029', '2034', '2046'],
    frequency=40,
    omega=[30],
    pde_name=['hetero'],
    pde_init_weight=1e-18,
    pde_loss_weight=1e-16,
    a_loss_weight=[0, 1e-4, 1e-2, 1e0],
    anatomical=True,
    polar_input=1,
)

len(param_space)

620

In [37]:
%autoreload

expt_name = '2023-2-16_patient_anat' # a_loss_weight

#jobs = ps.submit(template, name, queued_jobs, work_dir=expt_name, verbose=True)
#jobs.to_csv(f'{expt_name}.jobs')

#jobs = pd.read_csv(f'{expt_name}.jobs')

In [38]:
jobs

,Unnamed: 0.1,job_name,Unnamed: 0,data_name,xarray_dir,example_id,frequency,omega,pde_name,pde_init_weight,...,runtime_x,work_dir_x,array_idx_x,job_id_y,partition_y,job_state_y,node_id_y,runtime_y,work_dir_y,array_idx_y
0,0,train_patient_0717_30_hetero_1e-04,189,patient,../../../data/NAFLD/v4,717,40,30,hetero,1.000000e-18,...,0:00,/ocean/projects/asc170022p/mtragoza/mre-pinn/M...,NaN,14548069,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/mre-pinn/M...,NaN
1,1,train_patient_0717_30_hetero_1e-02,190,patient,../../../data/NAFLD/v4,717,40,30,hetero,1.000000e-18,...,0:00,/ocean/projects/asc170022p/mtragoza/mre-pinn/M...,NaN,14548070,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/mre-pinn/M...,NaN
2,2,train_patient_0717_30_hetero_1e+00,191,patient,../../../data/NAFLD/v4,717,40,30,hetero,1.000000e-18,...,0:00,/ocean/projects/asc170022p/mtragoza/mre-pinn/M...,NaN,14548071,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/mre-pinn/M...,NaN
3,3,train_patient_0727_30_hetero_0e+00,192,patient,../../../data/NAFLD/v4,727,40,30,hetero,1.000000e-18,...,0:00,/ocean/projects/asc170022p/mtragoza/mre-pinn/M...,NaN,14548072,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/mre-pinn/M...,NaN
4,4,train_patient_0727_30_hetero_1e-04,193,patient,../../../data/NAFLD/v4,727,40,30,hetero,1.000000e-18,...,0:00,/ocean/projects/asc170022p/mtragoza/mre-pinn/M...,NaN,14548073,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/mre-pinn/M...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426,426,train_patient_2034_30_hetero_1e+00,615,patient,../../../data/NAFLD/v4,2034,40,30,hetero,1.000000e-18,...,0:00,/ocean/projects/asc170022p/mtragoza/mre-pinn/M...,NaN,14548495,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/mre-pinn/M...,NaN
427,427,train_patient_2046_30_hetero_0e+00,616,patient,../../../data/NAFLD/v4,2046,40,30,hetero,1.000000e-18,...,0:00,/ocean/projects/asc170022p/mtragoza/mre-pinn/M...,NaN,14548496,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/mre-pinn/M...,NaN
428,428,train_patient_2046_30_hetero_1e-04,617,patient,../../../data/NAFLD/v4,2046,40,30,hetero,1.000000e-18,...,0:00,/ocean/projects/asc170022p/mtragoza/mre-pinn/M...,NaN,14548497,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/mre-pinn/M...,NaN
429,429,train_patient_2046_30_hetero_1e-02,618,patient,../../../data/NAFLD/v4,2046,40,30,hetero,1.000000e-18,...,0:00,/ocean/projects/asc170022p/mtragoza/mre-pinn/M...,NaN,14548498,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/mre-pinn/M...,NaN


In [28]:
job_ids = [
    14528598, 14528599, 14528600, 14528601, 14528602, 14528603, 14528604, 14528605, 14528606, 14528607, 14528608, 14528609, 14528610, 14528611, 14528612, 14528613, 14528614, 14528615, 14528616, 14528617, 14528618, 14528619, 14528620, 14528621, 14528622, 14528623, 14528624, 14528625, 14528626, 14528627, 14528628, 14528629, 14528630, 14528631, 14528632, 14528633, 14528634, 14528635, 14528636, 14528637, 14528638, 14528639, 14528640, 14528641, 14528642, 14528643, 14528644, 14528645, 14528646, 14528647, 14528648, 14528649, 14528650, 14528651, 14528652, 14528653, 14528654, 14528655, 14528656, 14528657, 14528658, 14528659, 14528660, 14528661, 14528662, 14528663, 14528664, 14528665, 14528666, 14528667, 14528668, 14528669, 14528670, 14528671, 14528672, 14528673, 14528674, 14528675, 14528676, 14528677, 14528678, 14528679, 14528680, 14528681, 14528682, 14528683, 14528684, 14528685, 14528686, 14528687, 14528688, 14528689, 14528690, 14528691, 14528692, 14528693, 14528694, 14528695, 14528696, 14528697, 14528698, 14528699, 14528700, 14528701, 14528702, 14528703, 14528704, 14528705, 14528706, 14528707, 14528708, 14528709, 14528710, 14528711, 14528712, 14528713, 14528714, 14528715, 14528716, 14528717, 14528718, 14528719, 14528720, 14528721, 14528722, 14528723, 14528724, 14528725, 14528726, 14528727, 14528728, 14528729, 14528730, 14528731, 14528732, 14528733, 14528734, 14528735, 14528736, 14528737, 14528738, 14528739, 14528740, 14528741, 14528742, 14528743, 14528744, 14528745, 14528746, 14528747, 14528748, 14528749, 14528750, 14528751, 14528752, 14528753, 14528754, 14528755, 14528756, 14528757, 14528758, 14528759, 14528760, 14528761, 14528762, 14528763, 14528764, 14528765, 14528766, 14528767, 14528768, 14528769, 14528770, 14528771, 14528772, 14528773, 14528774, 14528775, 14528776, 14528777, 14528778, 14528779, 14528780, 14528781, 14528782, 14528783, 14528784, 14528785, 14528786, 14528787, 14528788, 14528789, 14528790, 14528791, 14528792, 14528793, 14528794, 14528795, 14528796, 14528797, 14528798, 14528799, 14528800, 14528801, 14528802, 14528803, 14528804, 14528805, 14528806, 14528807, 14528808, 14528809, 14528810, 14528811, 14528812, 14528813, 14528814, 14528815, 14528816, 14528817, 14528818, 14528819, 14528820, 14528821, 14528822, 14528823, 14528824, 14528825, 14528826, 14528827, 14528828, 14528829, 14528830, 14528831, 14528832, 14528833, 14528834, 14528835, 14528836, 14528837, 14528838, 14528839, 14528840, 14528841, 14528842, 14528843, 14528844, 14528845, 14528846, 14528847, 14528848, 14528849, 14528850, 14528851, 14528852, 14528853, 14528854, 14528855, 14528856, 14528857, 14528858, 14528859, 14528860, 14528861, 14528862, 14528863, 14528865, 14528866, 14528867, 14528868, 14528869, 14528870, 14528871, 14528872, 14528873, 14528874, 14528875, 14528876, 14528877, 14528878, 14528879, 14528880, 14528881, 14528882, 14528883, 14528884, 14528885, 14528886, 14528887, 14528888, 14528889, 14528890, 14528891, 14528892, 14528893, 14528894, 14528895, 14528896, 14528897, 14528898, 14528899, 14528900, 14528901, 14528902, 14528903, 14528904, 14528905, 14528906, 14528907, 14528908, 14528909, 14528910, 14528911, 14528912, 14528913, 14528914, 14528915, 14528916, 14528917, 14528918, 14528919, 14528920, 14528921, 14528922, 14528923, 14528924, 14528925, 14528926, 14528927, 14528928, 14528929, 14528930, 14528931, 14528932, 14528933, 14528934, 14528935, 14528936, 14528937, 14528938, 14528939, 14528940, 14528941, 14528942, 14528943, 14528944, 14528945, 14528946, 14528947, 14528948, 14528949, 14528950, 14528951, 14528952, 14528953, 14528954, 14528955, 14528956, 14528957, 14528958, 14528959, 14528960, 14528961, 14528962, 14528963, 14528964, 14528965, 14528966, 14528967, 14528968, 14528969, 14528970, 14528971, 14528972, 14528973, 14528974, 14528975, 14528976, 14528977, 14528978, 14528979, 14528980, 14528981, 14528982, 14528983, 14528984, 14528985, 14528986, 14528987, 14528988, 14528989, 14528990, 14528991, 14528992, 14528993, 14528994, 14528995, 14528996, 14528997, 14528998, 14528999, 14529000, 14529001, 14529002, 14529003, 14529004, 14529005, 14529006, 14529007, 14529008, 14529009, 14529010, 14529011, 14529012, 14529013, 14529014, 14529015, 14529016, 14529017, 14529018, 14529019, 14529020, 14529021, 14529022, 14529023, 14529024, 14529025, 14529026, 14529027, 14529028, 14529029, 14529030, 14529031, 14529032, 14529033, 14529034, 14529035, 14529036, 14529037, 14529038, 14529039, 14529040, 14529041, 14529042, 14529043, 14529044, 14529045, 14529046, 14529047, 14529048, 14529049, 14529050, 14529051, 14529052, 14529053, 14529054, 14529055, 14529056, 14529057, 14529058, 14529059, 14529060, 14529061, 14529062, 14529063, 14529064, 14529065, 14529066, 14529067, 14529068, 14529069, 14529070, 14529071, 14529072, 14529073, 14529074, 14529075, 14529076, 14529077, 14529078, 14529079, 14529080, 14529081, 14529082, 14529083, 14529084, 14529085, 14529086, 14529087, 14529088, 14529089, 14529090, 14529091, 14529092, 14529093, 14529094, 14529095, 14529096, 14529097, 14529098, 14529099, 14529100, 14529101, 14529102, 14529103, 14529104, 14529105, 14529106, 14529107, 14529108, 14529109, 14529110, 14529111, 14529112, 14529113, 14529114, 14529115, 14529116, 14529117, 14529118, 14529119, 14529120, 14529121, 14529122, 14529123, 14529124, 14529125, 14529126, 14529127, 14529128, 14529129, 14529130, 14529131, 14529132, 14529133, 14529134, 14529135, 14529136, 14529137, 14529138, 14529139, 14529140, 14529141, 14529142, 14529143, 14529144, 14529145, 14529146, 14529147, 14529148, 14529149, 14529150, 14529151, 14529152, 14529153, 14529154, 14529155, 14529156, 14529157, 14529158, 14529159, 14529160, 14529161, 14529162, 14529163, 14529164, 14529165, 14529166, 14529167, 14529168, 14529169, 14529170, 14529171, 14529172, 14529173, 14529174, 14529175, 14529176, 14529177, 14529178, 14529179, 14529180, 14529181, 14529182, 14529183, 14529184, 14529185, 14529186, 14529187, 14529188, 14529189, 14529190, 14529191, 14529192, 14529193, 14529194, 14529195, 14529196, 14529197, 14529198, 14529199, 14529200, 14529201, 14529202, 14529203, 14529204, 14529205, 14529206, 14529207, 14529208, 14529209, 14529210, 14529211, 14529212, 14529213, 14529214, 14529215, 14529216, 14529217, 14529218
] + [
    14548069, 14548070, 14548071, 14548072, 14548073, 14548074, 14548075, 14548076, 14548077, 14548078, 14548079, 14548080, 14548081, 14548082, 14548083, 14548084, 14548085, 14548086, 14548087, 14548088, 14548089, 14548090, 14548091, 14548092, 14548093, 14548094, 14548095, 14548096, 14548097, 14548098, 14548099, 14548100, 14548101, 14548102, 14548103, 14548104, 14548105, 14548106, 14548107, 14548108, 14548109, 14548110, 14548111, 14548112, 14548113, 14548114, 14548115, 14548116, 14548117, 14548118, 14548119, 14548120, 14548121, 14548122, 14548123, 14548124, 14548125, 14548126, 14548127, 14548128, 14548129, 14548130, 14548131, 14548132, 14548133, 14548134, 14548135, 14548136, 14548137, 14548138, 14548139, 14548140, 14548141, 14548142, 14548143, 14548144, 14548145, 14548146, 14548147, 14548148, 14548149, 14548150, 14548151, 14548152, 14548153, 14548154, 14548155, 14548156, 14548157, 14548158, 14548159, 14548160, 14548161, 14548162, 14548163, 14548164, 14548165, 14548166, 14548167, 14548168, 14548169, 14548170, 14548171, 14548172, 14548173, 14548174, 14548175, 14548176, 14548177, 14548178, 14548179, 14548180, 14548181, 14548182, 14548183, 14548184, 14548185, 14548186, 14548187, 14548188, 14548189, 14548190, 14548191, 14548192, 14548193, 14548194, 14548195, 14548196, 14548197, 14548198, 14548199, 14548200, 14548201, 14548202, 14548203, 14548204, 14548205, 14548206, 14548207, 14548208, 14548209, 14548210, 14548211, 14548212, 14548213, 14548214, 14548215, 14548216, 14548217, 14548218, 14548219, 14548220, 14548221, 14548222, 14548223, 14548224, 14548225, 14548226, 14548227, 14548228, 14548229, 14548230, 14548231, 14548232, 14548233, 14548234, 14548235, 14548236, 14548237, 14548238, 14548239, 14548240, 14548241, 14548242, 14548243, 14548244, 14548245, 14548246, 14548247, 14548248, 14548249, 14548250, 14548251, 14548252, 14548253, 14548254, 14548255, 14548256, 14548257, 14548258, 14548259, 14548260, 14548261, 14548262, 14548263, 14548264, 14548265, 14548266, 14548267, 14548268, 14548269, 14548270, 14548271, 14548272, 14548273, 14548274, 14548275, 14548276, 14548277, 14548278, 14548279, 14548280, 14548281, 14548282, 14548283, 14548284, 14548285, 14548286, 14548287, 14548288, 14548289, 14548290, 14548291, 14548292, 14548293, 14548294, 14548295, 14548296, 14548297, 14548298, 14548299, 14548300, 14548301, 14548302, 14548303, 14548304, 14548305, 14548306, 14548307, 14548308, 14548309, 14548310, 14548311, 14548312, 14548313, 14548314, 14548315, 14548316, 14548317, 14548318, 14548319, 14548320, 14548321, 14548322, 14548323, 14548324, 14548325, 14548326, 14548327, 14548328, 14548329, 14548330, 14548331, 14548332, 14548333, 14548334, 14548335, 14548336, 14548337, 14548338, 14548339, 14548340, 14548341, 14548342, 14548343, 14548344, 14548345, 14548346, 14548347, 14548348, 14548349, 14548350, 14548351, 14548352, 14548353, 14548354, 14548355, 14548356, 14548357, 14548358, 14548359, 14548360, 14548361, 14548362, 14548363, 14548364, 14548365, 14548366, 14548367, 14548368, 14548369, 14548370, 14548371, 14548372, 14548373, 14548374, 14548375, 14548376, 14548377, 14548378, 14548379, 14548380, 14548381, 14548382, 14548383, 14548384, 14548385, 14548386, 14548387, 14548388, 14548389, 14548390, 14548391, 14548392, 14548393, 14548394, 14548395, 14548396, 14548397, 14548398, 14548399, 14548400, 14548401, 14548402, 14548403, 14548404, 14548405, 14548406, 14548407, 14548408, 14548409, 14548410, 14548411, 14548412, 14548413, 14548414, 14548415, 14548416, 14548417, 14548418, 14548419, 14548420, 14548421, 14548422, 14548423, 14548424, 14548425, 14548426, 14548427, 14548428, 14548429, 14548430, 14548431, 14548432, 14548433, 14548434, 14548435, 14548436, 14548437, 14548438, 14548439, 14548440, 14548441, 14548442, 14548443, 14548444, 14548445, 14548446, 14548447, 14548448, 14548449, 14548450, 14548451, 14548452, 14548453, 14548454, 14548455, 14548456, 14548457, 14548458, 14548459, 14548460, 14548461, 14548462, 14548463, 14548464, 14548465, 14548466, 14548467, 14548468, 14548469, 14548470, 14548471, 14548472, 14548473, 14548474, 14548475, 14548476, 14548477, 14548478, 14548479, 14548480, 14548481, 14548482, 14548483, 14548484, 14548485, 14548486, 14548487, 14548488, 14548489, 14548490, 14548491, 14548492, 14548493, 14548494, 14548495, 14548496, 14548497, 14548498, 14548499
]
print(len(job_ids))
ps.queue('slurm').get_job_status(job_ids)

1051


,job_name,job_id,partition,job_state,node_id,runtime,work_dir,array_idx
0,train_patient_0747_30_hetero_1e+00,14548091,GPU-shared,PENDING,(Priority),0:00,/ocean/projects/asc170022p/mtragoza/mre-pinn/M...,NaN
1,train_patient_0747_30_hetero_1e-04,14548089,GPU-shared,PENDING,(Priority),0:00,/ocean/projects/asc170022p/mtragoza/mre-pinn/M...,NaN
2,train_patient_0747_30_hetero_1e-02,14548090,GPU-shared,PENDING,(Priority),0:00,/ocean/projects/asc170022p/mtragoza/mre-pinn/M...,NaN
3,train_patient_0747_30_hetero_0e+00,14548088,GPU-shared,PENDING,(Priority),0:00,/ocean/projects/asc170022p/mtragoza/mre-pinn/M...,NaN
4,train_patient_0740_30_hetero_1e+00,14548087,GPU-shared,PENDING,(Priority),0:00,/ocean/projects/asc170022p/mtragoza/mre-pinn/M...,NaN
...,...,...,...,...,...,...,...,...
426,train_patient_0748_30_hetero_1e-04,14548093,GPU-shared,PENDING,(Priority),0:00,/ocean/projects/asc170022p/mtragoza/mre-pinn/M...,NaN
427,train_patient_0748_30_hetero_0e+00,14548092,GPU-shared,PENDING,(Priority),0:00,/ocean/projects/asc170022p/mtragoza/mre-pinn/M...,NaN
428,train_patient_0717_30_hetero_1e-04,14548069,GPU-shared,RUNNING,v032,4:58,/ocean/projects/asc170022p/mtragoza/mre-pinn/M...,NaN
429,train_patient_0717_30_hetero_1e-02,14548070,GPU-shared,RUNNING,v032,4:58,/ocean/projects/asc170022p/mtragoza/mre-pinn/M...,NaN


In [41]:
old_jobs = list(param_space)
old_job_files = ps.setup(template, name, old_jobs, work_dir=expt_name, dry_run=True)
old_job_ids = [
    14528598, 14528599, 14528600, 14528601, 14528602, 14528603, 14528604, 14528605, 14528606, 14528607, 14528608, 14528609, 14528610, 14528611, 14528612, 14528613, 14528614, 14528615, 14528616, 14528617, 14528618, 14528619, 14528620, 14528621, 14528622, 14528623, 14528624, 14528625, 14528626, 14528627, 14528628, 14528629, 14528630, 14528631, 14528632, 14528633, 14528634, 14528635, 14528636, 14528637, 14528638, 14528639, 14528640, 14528641, 14528642, 14528643, 14528644, 14528645, 14528646, 14528647, 14528648, 14528649, 14528650, 14528651, 14528652, 14528653, 14528654, 14528655, 14528656, 14528657, 14528658, 14528659, 14528660, 14528661, 14528662, 14528663, 14528664, 14528665, 14528666, 14528667, 14528668, 14528669, 14528670, 14528671, 14528672, 14528673, 14528674, 14528675, 14528676, 14528677, 14528678, 14528679, 14528680, 14528681, 14528682, 14528683, 14528684, 14528685, 14528686, 14528687, 14528688, 14528689, 14528690, 14528691, 14528692, 14528693, 14528694, 14528695, 14528696, 14528697, 14528698, 14528699, 14528700, 14528701, 14528702, 14528703, 14528704, 14528705, 14528706, 14528707, 14528708, 14528709, 14528710, 14528711, 14528712, 14528713, 14528714, 14528715, 14528716, 14528717, 14528718, 14528719, 14528720, 14528721, 14528722, 14528723, 14528724, 14528725, 14528726, 14528727, 14528728, 14528729, 14528730, 14528731, 14528732, 14528733, 14528734, 14528735, 14528736, 14528737, 14528738, 14528739, 14528740, 14528741, 14528742, 14528743, 14528744, 14528745, 14528746, 14528747, 14528748, 14528749, 14528750, 14528751, 14528752, 14528753, 14528754, 14528755, 14528756, 14528757, 14528758, 14528759, 14528760, 14528761, 14528762, 14528763, 14528764, 14528765, 14528766, 14528767, 14528768, 14528769, 14528770, 14528771, 14528772, 14528773, 14528774, 14528775, 14528776, 14528777, 14528778, 14528779, 14528780, 14528781, 14528782, 14528783, 14528784, 14528785, 14528786, 14528787, 14528788, 14528789, 14528790, 14528791, 14528792, 14528793, 14528794, 14528795, 14528796, 14528797, 14528798, 14528799, 14528800, 14528801, 14528802, 14528803, 14528804, 14528805, 14528806, 14528807, 14528808, 14528809, 14528810, 14528811, 14528812, 14528813, 14528814, 14528815, 14528816, 14528817, 14528818, 14528819, 14528820, 14528821, 14528822, 14528823, 14528824, 14528825, 14528826, 14528827, 14528828, 14528829, 14528830, 14528831, 14528832, 14528833, 14528834, 14528835, 14528836, 14528837, 14528838, 14528839, 14528840, 14528841, 14528842, 14528843, 14528844, 14528845, 14528846, 14528847, 14528848, 14528849, 14528850, 14528851, 14528852, 14528853, 14528854, 14528855, 14528856, 14528857, 14528858, 14528859, 14528860, 14528861, 14528862, 14528863, 14528865, 14528866, 14528867, 14528868, 14528869, 14528870, 14528871, 14528872, 14528873, 14528874, 14528875, 14528876, 14528877, 14528878, 14528879, 14528880, 14528881, 14528882, 14528883, 14528884, 14528885, 14528886, 14528887, 14528888, 14528889, 14528890, 14528891, 14528892, 14528893, 14528894, 14528895, 14528896, 14528897, 14528898, 14528899, 14528900, 14528901, 14528902, 14528903, 14528904, 14528905, 14528906, 14528907, 14528908, 14528909, 14528910, 14528911, 14528912, 14528913, 14528914, 14528915, 14528916, 14528917, 14528918, 14528919, 14528920, 14528921, 14528922, 14528923, 14528924, 14528925, 14528926, 14528927, 14528928, 14528929, 14528930, 14528931, 14528932, 14528933, 14528934, 14528935, 14528936, 14528937, 14528938, 14528939, 14528940, 14528941, 14528942, 14528943, 14528944, 14528945, 14528946, 14528947, 14528948, 14528949, 14528950, 14528951, 14528952, 14528953, 14528954, 14528955, 14528956, 14528957, 14528958, 14528959, 14528960, 14528961, 14528962, 14528963, 14528964, 14528965, 14528966, 14528967, 14528968, 14528969, 14528970, 14528971, 14528972, 14528973, 14528974, 14528975, 14528976, 14528977, 14528978, 14528979, 14528980, 14528981, 14528982, 14528983, 14528984, 14528985, 14528986, 14528987, 14528988, 14528989, 14528990, 14528991, 14528992, 14528993, 14528994, 14528995, 14528996, 14528997, 14528998, 14528999, 14529000, 14529001, 14529002, 14529003, 14529004, 14529005, 14529006, 14529007, 14529008, 14529009, 14529010, 14529011, 14529012, 14529013, 14529014, 14529015, 14529016, 14529017, 14529018, 14529019, 14529020, 14529021, 14529022, 14529023, 14529024, 14529025, 14529026, 14529027, 14529028, 14529029, 14529030, 14529031, 14529032, 14529033, 14529034, 14529035, 14529036, 14529037, 14529038, 14529039, 14529040, 14529041, 14529042, 14529043, 14529044, 14529045, 14529046, 14529047, 14529048, 14529049, 14529050, 14529051, 14529052, 14529053, 14529054, 14529055, 14529056, 14529057, 14529058, 14529059, 14529060, 14529061, 14529062, 14529063, 14529064, 14529065, 14529066, 14529067, 14529068, 14529069, 14529070, 14529071, 14529072, 14529073, 14529074, 14529075, 14529076, 14529077, 14529078, 14529079, 14529080, 14529081, 14529082, 14529083, 14529084, 14529085, 14529086, 14529087, 14529088, 14529089, 14529090, 14529091, 14529092, 14529093, 14529094, 14529095, 14529096, 14529097, 14529098, 14529099, 14529100, 14529101, 14529102, 14529103, 14529104, 14529105, 14529106, 14529107, 14529108, 14529109, 14529110, 14529111, 14529112, 14529113, 14529114, 14529115, 14529116, 14529117, 14529118, 14529119, 14529120, 14529121, 14529122, 14529123, 14529124, 14529125, 14529126, 14529127, 14529128, 14529129, 14529130, 14529131, 14529132, 14529133, 14529134, 14529135, 14529136, 14529137, 14529138, 14529139, 14529140, 14529141, 14529142, 14529143, 14529144, 14529145, 14529146, 14529147, 14529148, 14529149, 14529150, 14529151, 14529152, 14529153, 14529154, 14529155, 14529156, 14529157, 14529158, 14529159, 14529160, 14529161, 14529162, 14529163, 14529164, 14529165, 14529166, 14529167, 14529168, 14529169, 14529170, 14529171, 14529172, 14529173, 14529174, 14529175, 14529176, 14529177, 14529178, 14529179, 14529180, 14529181, 14529182, 14529183, 14529184, 14529185, 14529186, 14529187, 14529188, 14529189, 14529190, 14529191, 14529192, 14529193, 14529194, 14529195, 14529196, 14529197, 14529198, 14529199, 14529200, 14529201, 14529202, 14529203, 14529204, 14529205, 14529206, 14529207, 14529208, 14529209, 14529210, 14529211, 14529212, 14529213, 14529214, 14529215, 14529216, 14529217, 14529218
]
old_jobs = pd.DataFrame(old_jobs)
old_jobs['work_dir'] = [os.path.dirname(f) for f in old_job_files]
old_jobs['job_id'] = old_job_ids
old_jobs

,data_name,xarray_dir,example_id,frequency,omega,pde_name,pde_init_weight,pde_loss_weight,a_loss_weight,anatomical,polar_input,job_name,work_dir,job_id
0,patient,../../../data/NAFLD/v4,0006,40,30,hetero,1.000000e-18,1.000000e-16,0.0000,True,1,train_patient_0006_30_hetero_0e+00,2023-2-16_patient_anat/train_patient_0006_30_h...,14528598
1,patient,../../../data/NAFLD/v4,0006,40,30,hetero,1.000000e-18,1.000000e-16,0.0001,True,1,train_patient_0006_30_hetero_1e-04,2023-2-16_patient_anat/train_patient_0006_30_h...,14528599
2,patient,../../../data/NAFLD/v4,0006,40,30,hetero,1.000000e-18,1.000000e-16,0.0100,True,1,train_patient_0006_30_hetero_1e-02,2023-2-16_patient_anat/train_patient_0006_30_h...,14528600
3,patient,../../../data/NAFLD/v4,0006,40,30,hetero,1.000000e-18,1.000000e-16,1.0000,True,1,train_patient_0006_30_hetero_1e+00,2023-2-16_patient_anat/train_patient_0006_30_h...,14528601
4,patient,../../../data/NAFLD/v4,0020,40,30,hetero,1.000000e-18,1.000000e-16,0.0000,True,1,train_patient_0020_30_hetero_0e+00,2023-2-16_patient_anat/train_patient_0020_30_h...,14528602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,patient,../../../data/NAFLD/v4,2034,40,30,hetero,1.000000e-18,1.000000e-16,1.0000,True,1,train_patient_2034_30_hetero_1e+00,2023-2-16_patient_anat/train_patient_2034_30_h...,14529214
616,patient,../../../data/NAFLD/v4,2046,40,30,hetero,1.000000e-18,1.000000e-16,0.0000,True,1,train_patient_2046_30_hetero_0e+00,2023-2-16_patient_anat/train_patient_2046_30_h...,14529215
617,patient,../../../data/NAFLD/v4,2046,40,30,hetero,1.000000e-18,1.000000e-16,0.0001,True,1,train_patient_2046_30_hetero_1e-04,2023-2-16_patient_anat/train_patient_2046_30_h...,14529216
618,patient,../../../data/NAFLD/v4,2046,40,30,hetero,1.000000e-18,1.000000e-16,0.0100,True,1,train_patient_2046_30_hetero_1e-02,2023-2-16_patient_anat/train_patient_2046_30_h...,14529217


In [42]:
ps.status(old_jobs)

,index,data_name,xarray_dir,example_id,frequency,omega,pde_name,pde_init_weight,pde_loss_weight,a_loss_weight,anatomical,polar_input,job_name,work_dir,job_state,node_id,runtime,stdout,stderr
job_id,,,,,,,,,,,,,,,,,,,
14528598,0,patient,../../../data/NAFLD/v4,0006,40,30,hetero,1.000000e-18,1.000000e-16,0.0000,True,1,train_patient_0006_30_hetero_0e+00,2023-2-16_patient_anat/train_patient_0006_30_h...,NaN,NaN,NaN,dv002.bridges2.psc.edu\n/ocean/projects/asc170...,Using backend: pytorch\n\nWARNING:py.warnings:...
14528599,1,patient,../../../data/NAFLD/v4,0006,40,30,hetero,1.000000e-18,1.000000e-16,0.0001,True,1,train_patient_0006_30_hetero_1e-04,2023-2-16_patient_anat/train_patient_0006_30_h...,NaN,NaN,NaN,dv003.bridges2.psc.edu\n/ocean/projects/asc170...,Using backend: pytorch\n\n
14528600,2,patient,../../../data/NAFLD/v4,0006,40,30,hetero,1.000000e-18,1.000000e-16,0.0100,True,1,train_patient_0006_30_hetero_1e-02,2023-2-16_patient_anat/train_patient_0006_30_h...,NaN,NaN,NaN,dv003.bridges2.psc.edu\n/ocean/projects/asc170...,Using backend: pytorch\n\n
14528601,3,patient,../../../data/NAFLD/v4,0006,40,30,hetero,1.000000e-18,1.000000e-16,1.0000,True,1,train_patient_0006_30_hetero_1e+00,2023-2-16_patient_anat/train_patient_0006_30_h...,NaN,NaN,NaN,dv003.bridges2.psc.edu\n/ocean/projects/asc170...,Using backend: pytorch\n\n
14528602,4,patient,../../../data/NAFLD/v4,0020,40,30,hetero,1.000000e-18,1.000000e-16,0.0000,True,1,train_patient_0020_30_hetero_0e+00,2023-2-16_patient_anat/train_patient_0020_30_h...,NaN,NaN,NaN,dv003.bridges2.psc.edu\n/ocean/projects/asc170...,Using backend: pytorch\n\nWARNING:py.warnings:...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14529214,615,patient,../../../data/NAFLD/v4,2034,40,30,hetero,1.000000e-18,1.000000e-16,1.0000,True,1,train_patient_2034_30_hetero_1e+00,2023-2-16_patient_anat/train_patient_2034_30_h...,NaN,NaN,NaN,NaN,NaN
14529215,616,patient,../../../data/NAFLD/v4,2046,40,30,hetero,1.000000e-18,1.000000e-16,0.0000,True,1,train_patient_2046_30_hetero_0e+00,2023-2-16_patient_anat/train_patient_2046_30_h...,NaN,NaN,NaN,NaN,NaN
14529216,617,patient,../../../data/NAFLD/v4,2046,40,30,hetero,1.000000e-18,1.000000e-16,0.0001,True,1,train_patient_2046_30_hetero_1e-04,2023-2-16_patient_anat/train_patient_2046_30_h...,NaN,NaN,NaN,NaN,NaN


In [45]:
old_jobs.to_csv(f'{expt_name}.jobs')

In [46]:
jobs = pd.read_csv(f'{expt_name}.jobs')

In [47]:
status_cols = ['job_name', 'job_state', 'node_id', 'runtime', 'stdout', 'stderr']
status = ps.status(jobs)
status[status_cols]

,job_name,job_state,node_id,runtime,stdout,stderr
job_id,,,,,,
14528598,train_patient_0006_30_hetero_0e+00,NaN,NaN,NaN,dv002.bridges2.psc.edu\n/ocean/projects/asc170...,Using backend: pytorch\n\nWARNING:py.warnings:...
14528599,train_patient_0006_30_hetero_1e-04,NaN,NaN,NaN,dv003.bridges2.psc.edu\n/ocean/projects/asc170...,Using backend: pytorch\n\n
14528600,train_patient_0006_30_hetero_1e-02,NaN,NaN,NaN,dv003.bridges2.psc.edu\n/ocean/projects/asc170...,Using backend: pytorch\n\n
14528601,train_patient_0006_30_hetero_1e+00,NaN,NaN,NaN,dv003.bridges2.psc.edu\n/ocean/projects/asc170...,Using backend: pytorch\n\n
14528602,train_patient_0020_30_hetero_0e+00,NaN,NaN,NaN,dv003.bridges2.psc.edu\n/ocean/projects/asc170...,Using backend: pytorch\n\nWARNING:py.warnings:...
...,...,...,...,...,...,...
14529214,train_patient_2034_30_hetero_1e+00,NaN,NaN,NaN,NaN,NaN
14529215,train_patient_2046_30_hetero_0e+00,NaN,NaN,NaN,NaN,NaN
14529216,train_patient_2046_30_hetero_1e-04,NaN,NaN,NaN,NaN,NaN


In [ ]:
errors = status[status.stderr != 'Using backend: pytorch\n\n']
len(errors)

In [ ]:
metrics = ps.metrics(jobs[jobs.pde_name != 'hetero2'])

# did all models train to 100k iterations?
#assert (metrics.groupby('job_name')['iteration'].max() == 100e3).all()

# get the final test evaluations
metrics = metrics[metrics.iteration == 100e3]

param_cols = ['pde_name', 'example_id', 'omega'] # experimental parameters
index_cols = ['variable_name', 'spatial_frequency_bin', 'spatial_region'] # metric identifiers
metric_cols = ['MSAV', 'PSD', 'MAV', 'R'] # metric values

metrics = metrics.groupby(param_cols + index_cols, sort=False)[metric_cols].mean()
metrics = metrics.unstack(level=[len(param_cols)])

def metric_map(t):
    metric_name, var_name = t
    new_col_name = f'{var_name}_{metric_name}'
    new_col_name = new_col_name.replace('diff_MSAV', 'pred_MSAE')
    new_col_name = new_col_name.replace('pde_diff_MSAV', 'PDE_MSAE')
    new_col_name = new_col_name.replace('diff_MAV', 'pred_MAD')
    return new_col_name

metrics.columns = [metric_map(t) for t in metrics.columns.to_flat_index()]
metrics

In [ ]:
m = metrics.reset_index()

#fig = ps.plot(
#    m[m.spatial_region == 'all'],
#    x=param_cols,
#    y=['u_pred_MSAE', 'mu_pred_MSAE', 'direct_pred_MSAE', 'fem_pred_MSAE'],
#    height=2.5,
#    width=2.5,
#    legend=False,
#    tight=True
#)

In [ ]:
m.columns

In [ ]:
m['method'] = 'PINN_' + m['pde_name']

direct_m = m[m.pde_name == 'helmholtz'].copy()
direct_m['method'] = 'direct_helmholtz'
for col in m.columns:
    if col.startswith('direct'):
        direct_m[col.replace('direct', 'mu')] = m[col]
        
hh_fem_m = m[m.pde_name == 'helmholtz'].copy()
hh_fem_m['method'] = 'FEM_helmholtz'
for col in m.columns:
    if col.startswith('fem'):
        hh_fem_m[col.replace('fem', 'mu')] = m[col]

ht_fem_m = m[m.pde_name == 'hetero'].copy()
ht_fem_m['method'] = 'FEM_hetero'
for col in m.columns:
    if col.startswith('fem'):
        ht_fem_m[col.replace('fem', 'mu')] = m[col]
        
mm = pd.concat([direct_m, hh_fem_m, ht_fem_m, m])

method_order = ['direct_helmholtz', 'FEM_helmholtz', 'FEM_hetero', 'PINN_helmholtz', 'PINN_hetero']
mm['method_index'] = mm['method'].map(method_order.index)
mm = mm.sort_values('method_index')

In [ ]:
import matplotlib as mpl
pct_format = mpl.ticker.PercentFormatter()

mm['mu_pred_MSAE_relative'] = mm['mu_pred_MSAE'] / mm['mu_true_MSAV'] * 100
mm['mu_pred_MAD_relative'] = mm['mu_pred_MAD'] / mm['mu_true_MAV'] * 100

In [ ]:
%autoreload
import seaborn as sns
import matplotlib.pyplot as plt

colors = ps.results.get_color_palette(10, type='deep', min_val=0)
blue, orange, green, red, purple, brown, pink, gray, yellow, cyan = colors

colors = [blue, yellow, red, cyan, green]
#colors = [blue, yellow, cyan]
sns.set_palette(colors)
colors = sns.color_palette()
colors

In [ ]:
mm.spatial_region.unique()

In [ ]:
fig = ps.plot(
    mm[~mm.spatial_region.isin({'0.0', 'all'})],
    x='method',
    y='mu_pred_MAD_relative',
    height=3,
    width=7,
    legend=True,
    tight=True,
    plot_func=ps.results.barplot
)
fig.axes[0].set_ylabel('$\mu$ relative MAD (%)')
fig.axes[0].set_ylim(0, 100)
fig.axes[0].yaxis.set_major_formatter(pct_format)
fig.tight_layout()

for i, patch in enumerate(fig.axes[0].patches):
    plt.setp(patch, facecolor=colors[i])   

for ext in ['png', 'pdf']:
    fig.savefig(f'images/patient_method_bar_plot.{ext}', bbox_inches='tight', dpi=200)

In [ ]:
fig = ps.plot(
    mm[(mm.spatial_region == 'all') & (mm.spatial_frequency_bin == 'all')].copy(), # across all omegas
    x='method',
    hue='method',
    y='mu_pred_R',
    height=3,
    width=7,
    legend=False,
    tight=True,
    plot_func=ps.results.barplot
)
fig.axes[0].set_ylabel('$\mu$ correlation (R)')
fig.axes[0].set_ylim([0, 1])

for i, patch in enumerate(fig.axes[0].patches):
    plt.setp(patch, facecolor=colors[i])

for ext in ['png', 'pdf']:
    fig.savefig(f'images/patient_method_R_bar_plot.{ext}', bbox_inches='tight', dpi=200)

In [ ]:
fig = ps.plot(
    mm[~mm.spatial_region.isin({'0', 'all'}) & mm.method.str.startswith('PINN')], 
    x='omega',
    y='mu_pred_MAD_relative',
    height=5,
    width=6,
    legend=True,
    tight=True,
    plot_func=ps.results.barplot
)
fig.axes[0].set_ylabel('$\mu$ relative MAD (%)')
fig.axes[0].yaxis.set_major_formatter(pct_format)
fig.axes[0].set_ylim(0, 100)
fig.tight_layout()

fig.axes[0].set_xlabel('omega')

for ext in ['png', 'pdf']:
    fig.savefig(f'images/patient_omega_bar_plot.{ext}', bbox_inches='tight', dpi=200)

In [ ]:
%autoreload

blue, orange, green, red, purple, brown, pink, gray, yellow, cyan = sns.color_palette('tab10')

mre_pinn.visual.COLORS.update(
    red=red,
    yellow=yellow,
    green=green,
    cyan=cyan,
    blue=blue
)

In [ ]:
example.anat.sizes

In [ ]:
example = mre_pinn.data.MREExample.load_xarrays('../data/NAFLD/v3', '0006', anat=True)
example.arrays['anat'] = example.anat.sel(z=0.018237183332, method='nearest')
example.arrays['anat_mask'] = example.anat_mask.sel(z=0.018237183332, method='nearest')
example.view('anat', ax_height=2, space=0.2)
plt.gcf().tight_layout()
plt.savefig('images/patient_anat.png', dpi=200, bbox_inches='tight')
example.view('anat_mask', ax_height=2, space=0.2)
plt.gcf().tight_layout()
plt.savefig('images/patient_mask.png', dpi=200, bbox_inches='tight')
example.view('wave', ax_height=2, space=0.2)
plt.gcf().tight_layout()
plt.savefig('images/patient_wave.png', dpi=200, bbox_inches='tight')
example.view('mre', ax_height=2, space=0.2)
plt.gcf().tight_layout()
plt.savefig('images/patient_mre.png', dpi=200, bbox_inches='tight')

In [ ]:
%autoreload
import mre_pinn

image_names = ['wave image', 'ground truth', 'direct_helmholtz', 'FEM_helmholtz', 'FEM_hetero', 'PINN_helmholtz', 'PINN_hetero']

def plot_image_grid(example_ids):
    n_rows = len(example_ids)
    n_cols = len(image_names)
    ax_width = 1.2
    ax_height = ax_width

    fig, axes, cbar_ax = mre_pinn.visual.subplot_grid(n_rows, n_cols, ax_height, ax_width, space=0, pad=(0.35,0.15,0.15,0.25))
    for row_idx, example_id in enumerate(example_ids):
        example = mre_pinn.data.MREExample.load_xarrays('../data/NAFLD/v3', example_id)
        for col_idx, image_name in enumerate(image_names):
            ax = axes[row_idx,col_idx]
            #ax.text(0.1, 0.1, f'{row_idx}, {col_idx}')
            if col_idx == 0:
                ax.set_ylabel(example_id, fontsize='medium')
            if row_idx == 0:
                ax.set_title(image_name, fontsize='small')
            ax.set_yticks([])
            ax.set_xticks([])

            if image_name == 'wave image':
                array = example.wave[...,0]
                color_kws = mre_pinn.visual.get_color_kws(array)
                color_kws['vmin'] = -color_kws['vmax']
                array = array.real * example.mre_mask.values[...,0]

            elif image_name == 'ground truth':
                array = example.mre[...,0]
                color_kws = mre_pinn.visual.get_color_kws(array)
                color_kws['vmin'] = -color_kws['vmax']
                array = np.abs(array) * example.mre_mask.values[...,0]

            elif image_name == 'direct_helmholtz':
                nc_file = f'2022-11-26_patient_hetero2/train_patient_{example_id}_helmholtz/train_patient_{example_id}_helmholtz_direct.nc'
                array = mre_pinn.data.dataset.load_xarray_file(nc_file).sel(variable='direct_pred')[...,0]
                color_kws = mre_pinn.visual.get_color_kws(array)
                color_kws['vmin'] = -color_kws['vmax']
                array = np.abs(array)

            elif image_name == 'FEM_helmholtz':
                nc_file = f'2022-11-26_patient_hetero2/train_patient_{example_id}_helmholtz/train_patient_{example_id}_helmholtz_fem.nc'
                array = mre_pinn.data.dataset.load_xarray_file(nc_file).sel(variable='fem_pred')[...,0]
                color_kws = mre_pinn.visual.get_color_kws(array)
                color_kws['vmin'] = -color_kws['vmax']
                array = np.abs(array)

            elif image_name == 'FEM_hetero':
                nc_file = f'2022-11-23_patient_init/train_patient_{example_id}_hetero/train_patient_{example_id}_hetero_fem.nc'
                array = mre_pinn.data.dataset.load_xarray_file(nc_file).sel(variable='fem_pred')[...,0]
                color_kws = mre_pinn.visual.get_color_kws(array)
                color_kws['vmin'] = -color_kws['vmax']
                array = np.abs(array)

            elif image_name == 'PINN_helmholtz':
                nc_file = f'2022-11-26_patient_hetero2/train_patient_{example_id}_helmholtz/train_patient_{example_id}_helmholtz_elastogram.nc'
                array = mre_pinn.data.dataset.load_xarray_file(nc_file).sel(variable='mu_pred')[...,0]
                color_kws = mre_pinn.visual.get_color_kws(array)
                color_kws['vmin'] = -color_kws['vmax']
                array = np.abs(array)

            elif image_name == 'PINN_hetero':
                nc_file = f'2022-11-23_patient_init/train_patient_{example_id}_hetero/train_patient_{example_id}_hetero_elastogram.nc'
                array = mre_pinn.data.dataset.load_xarray_file(nc_file).sel(variable='mu_pred')[...,0]
                color_kws = mre_pinn.visual.get_color_kws(array)
                color_kws['vmin'] = -color_kws['vmax']
                array = np.abs(array)

            mre_pinn.visual.imshow(ax, array[:-24,8:-16], **color_kws)
    
    return fig

fig0 = plot_image_grid(['0006', '0020', '0024', '0029'])
fig1 = plot_image_grid(['0043', '0047', '0126', '0135'])

for ext in ['png', 'pdf']:
    fig0.savefig(f'images/patient_example_grid0.{ext}', bbox_inches='tight', dpi=200)
    fig1.savefig(f'images/patient_example_grid1.{ext}', bbox_inches='tight', dpi=200)

In [ ]:
example = mre_pinn.data.MREExample.load_xarrays('../data/NAFLD/v3', '0006')
example.view()

In [ ]:
import xarray as xr
u = example.wave
x = u.field.points(reshape=False)
x = (x - x.mean(axis=(0,1,2), keepdims=True)) / (x.max(axis=(0,1,2), keepdims=True) - x.min(axis=(0,1,2), keepdims=True))
x, y, z = x[...,0], x[...,1], x[...,2]
r = np.sqrt(x**2 + y**2)
sin = y / (r + 1e-3)
cos = x / (r + 1e-3)
x = np.stack([x, y, z, r, sin, cos], axis=-1)

print(x.shape)
x_dims = ['x', 'y', 'z', 'component']
x_coords = {
    'x': u.x,
    'y': u.y,
    'z': u.z,
    'component': ['x', 'y', 'z', 'r', 'sin', 'cos']
}
x = xr.DataArray(x, coords=x_coords, dims=x_dims)
x.name = 'position'
mre_pinn.visual.XArrayViewer(x, ax_height=3)

In [ ]:
example.wave.coords